[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/SD-CN-Animation-colab/blob/main/SD-CN-Animation-colab.ipynb)

In [ ]:
!git clone https://huggingface.co/camenduru/SD-CN-Animation